# Thriller Game Agent

## Description

A narrative-driven thriller game powered by AI agents, where players interact with a dynamic Narrator Agent that weaves suspenseful storylines — and secretly consults a Research Agent for accurate real-world facts when needed.

## Install the necessary python libraries and load the OpenAI API Key

In [1]:
!pip -q install openai-agents

In [4]:
from dotenv import load_dotenv
import os

# Load .env file
load_dotenv(dotenv_path='./resources/openaiApiKey.env')

# Retrieve the key
api_key = os.getenv('OPENAI_API_KEY')

print(f"API Key loaded: {bool(api_key)}")

API Key loaded: True


In [50]:
# Import agent libraries
from agents import Agent, Runner, function_tool, FileSearchTool, WebSearchTool

from typing import Literal

In [2]:
# Add the nest_asyncio and asyncio to allow for async operations in Jupyter Notesbooks
import nest_asyncio
nest_asyncio.apply()
import asyncio

## Game Setup

In [5]:
# The basic plot and story of the game.
GAME_STORY = """

**Setting:**  
The near future. Technology is slightly advanced, but society still resembles today’s world.

**Player Character:**  
- A citizen born with a rare genetic trait that grants an extremely extended lifespan (potentially up to 1000 years).
- Currently living a quiet, anonymous life in Queens, New York.
- They are unaware that their gene is highly sought after.

**Antagonist:**  
- An aging billionaire, now in declining health.
- Obsessed with longevity, they have spent decades searching for someone with this rare gene.
- They have secretly deployed private agents to capture the player for experimentation or direct blood transfusion.

**Story Start:**  
The player is awoken in the middle of the night. Strange noises come from outside. Agents have found them. The player must escape, survive, and find safety while uncovering the truth behind this conspiracy.

**Tone:**  
Thriller. Suspenseful. High-stakes cat-and-mouse. Always keep the tension alive. Describe situations vividly. Responses should be immersive, cinematic, and detailed.

**Gameplay:**  
- Respond to the player’s actions one step at a time.
- Offer vivid descriptions of the environment and consequences.
- Encourage creative problem-solving.
- Do not make decisions for the player — allow them to lead.

**Example Prompt-Response Flow:**
Player: "I look out the window."
AI: "You cautiously peek through the blinds. A black SUV idles outside. Two figures in suits approach your door, speaking into earpieces. They’re not here for a friendly visit."



"""

In [47]:
# Log the current details of the game.
GAME_LOG = []

In [48]:
# Log the current details of the game.
RESEARCH_LOG = []

In [9]:
# A list of items that the player has access to
PLAYER_ITEMS = []

## Set up the Function Tools

Parameter types for function tools must be explicitly defined.

In [45]:
@function_tool
async def update_game_log(new_entry: str, category: Literal["event", "discovery", "decision", "question", "item", "ambient"] = "event") -> str:
    """
    Saves a structured log entry to the game log with a category.
    """
    global GAME_LOG
    GAME_LOG.append({
        "category": category,
        "entry": new_entry
    })
    return f"Game log updated with a {category} entry."

In [51]:
@function_tool
async def update_research_log(new_entry: str, category: Literal["info", "symbol", "historical", "technical", "psychological", "warning"] = "info") -> str:
    """
    Saves a structured log entry to the research log with a category.
    """
    global RESEARCH_LOG
    RESEARCH_LOG.append({
        "category": category,
        "entry": new_entry
    })
    return f"Research log updated with a {category} entry."

In [52]:
@function_tool
async def add_player_item(item_name: str, description: str = "") -> str:
    """
    Adds a new item to the player's inventory.
    Each item is stored as a dictionary with a name and optional description.
    """
    global PLAYER_ITEMS
    # Prevent duplicate item names
    if any(item["name"] == item_name for item in PLAYER_ITEMS):
        return f"{item_name} is already in your inventory."

    PLAYER_ITEMS.append({
        "name": item_name,
        "description": description
    })
    return f"{item_name} added to your inventory."


In [53]:
@function_tool
async def remove_player_item(item_name: str) -> str:
    """
    Removes an item from the player's inventory by name.
    """
    global PLAYER_ITEMS
    for item in PLAYER_ITEMS:
        if item["name"] == item_name:
            PLAYER_ITEMS.remove(item)
            return f"{item_name} removed from your inventory."

    return f"{item_name} not found in your inventory."


In [27]:
@function_tool
async def query_web_research_agent(query: str) -> str:
    """
    Allows the narrator agent to query the Web Research Agent for factual information.
    This tool is only for the narrator's use — the player should never see it being called.
    """
    coroutine = Runner.run(web_research_agent, query)
    response = await coroutine
    return response

## Setup and Test the Agents

### Create the Web Research Agent

In [23]:
web_research_agent = Agent(name="Web Research Agent",
                instructions=f"""
                You are a research agent that supports the Thriller Narrator Agent.
                The Narrator Agent does not have access to the internet, but sometimes requires outside information to support its narrator role.
                You expertly fulfill this narrative support role by giving the narrator agent accurate and succinct answers to its questions.
                Save all research queries and results to the {RESEARCH_LOG}""",
                model="gpt-4o",
                tools=[WebSearchTool(),update_research_log]
                )

### Test Web Research Agent

Runner.run(...) returns a coroutine object (not a regular return value), so it must be awaited manually.

In [25]:
coroutine = Runner.run(web_research_agent, "Is it raining in France today?")
result = await coroutine
print(result.final_output)

<class 'coroutine'>
France is mostly experiencing dry conditions today, though some regions in the north and east, like Lille, Strasbourg, and Breux, may see showers or thunderstorms. Paris remains mostly cloudy without rain.


### Test the Research Log

In [28]:
print(RESEARCH_LOG)


Research log starts here:


Weather in France on October 5, 2023: Overall dry and warm across major cities such as Paris, Lyon, Marseille, Toulouse, and Bordeaux. No significant rainfall reported.

Weather in France on August 2, 2025. Mostly dry conditions in France today. Paris: Mostly cloudy, no rain. Lille: Mostly cloudy, possible passing showers, Yellow Warning for Thunderstorms. Strasbourg and Breux: Mostly cloudy, few showers expected, Thunderstorm warning. Other cities like Lyon, Marseille, Toulouse have pleasant weather with no rain expected.



### Create the Narrator Agent

In [61]:
narrator_agent = Agent(
    name="Thriller Narrator Agent",
    instructions=f"""
        You are the narrator and game master for a text-based thriller set in the near future.
        The player will interact with the world using simple text commands.
        Your role is to describe scenes, characters, tension, and consequences vividly, and to guide the player through this interactive experience in a grounded and immersive way.
        
        The game world is described here:
        {GAME_STORY}
        
        Current game details are stored here:
        {GAME_LOG}
        
        The player’s current inventory is listed here:
        {PLAYER_ITEMS}
        
        After describing the outcome of each player action:
        
        - You **must** use the `update_game_log` tool to record key narrative events and progress updates.
          - Log only meaningful, concise entries — such as player choices, discoveries, gained or lost items, or critical moments.
          - Do **not** repeat the full narration in the log.
          - Suggested log categories include: "event", "discovery", "decision", "question", "item", or "ambient".
        
        - If the player **picks up, finds, receives, or otherwise gains an item**, you **must** call the `add_player_item` tool **immediately** after narration.
          - Use the exact item name mentioned in the story.
          - Include a short description (e.g., "a heavy clown shoe with blood on the heel").
        
        - If the player **drops, discards, loses, or destroys an item**, you **must** call the `remove_player_item` tool with the correct item name.
        
        - Do **not** explain or mention tool usage to the player. Tool calls happen invisibly, behind the scenes.
        
        Maintain suspense, tension, and atmospheric detail — but always remember to keep the game log and inventory accurate and up to date using the tools provided.
        """,
    model="gpt-4o",
    tools=[
        update_game_log,
        add_player_item,
        remove_player_item,
        query_web_research_agent
    ]
)


### Test the Narrator Agent

In [ ]:
coroutine = Runner.run(narrator_agent, "Where am I?")
result = await coroutine
print(result.final_output)

### Test the Game Log

In [42]:
coroutine = Runner.run(narrator_agent, "I run outside straight at the SUV")
result = await coroutine
print(result.final_output)

The figure in the suit seems poised for any sudden move. What's your next move?


Nothing should be saved here because the player input was insignificant.

In [43]:
print(GAME_LOG)


Begin the game:


Player runs straight at the SUV. A figure in a suit emerges, appearing surprised and cautious.



In [ ]:
coroutine = Runner.run(narrator_agent, "I run straight at the SUV")
result = await coroutine
print(result.final_output)

With this updated information, the Game Log will now have updates populated

In [44]:
print(GAME_LOG)


Begin the game:


Player runs straight at the SUV. A figure in a suit emerges, appearing surprised and cautious.



## Test the inventory update system

In [67]:
coroutine = Runner.run(narrator_agent, "I pick up a bright flashlight.")
result = await coroutine
print(result.final_output)

The flashlight feels solid, its beam cutting through the dim light of the room. Outside, footsteps draw closer. Your heart races. What's your next move?


In [68]:
print(PLAYER_ITEMS)

[{'name': 'bright flashlight', 'description': 'a sturdy and reliable flashlight with a strong beam'}]


In [69]:
coroutine = Runner.run(narrator_agent, "I tripped and dropped the bright flashlight.")
result = await coroutine
print(result.final_output)

The flashlight has slipped from your grasp and now lies on the floor, its light still flickering into the shadows. What will you do next?


In [70]:
print(PLAYER_ITEMS)

[]
